# Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpl
from sklearn.datasets import make_classification
from scipy.stats import norm

# Load Data

In [ ]:
class DataGenerator:
    def __init__(self, coef_):
        self.coef_ = coef_

    def next(self, N=100):
        x0 = np.full((1, N), self.coef_[0])
        x1 = np.random.normal(self.coef_[1], 0.1, N)
        x2 = np.random.normal(0.3 * x1, 0.1, N)
        x3 = np.random.normal(0.25 * x1, 0.1, N)
        x4 = np.random.normal(0.25 * x2 + 0.33 * x3, 0.2, N)
        x5 = np.random.normal(0.77 * x4, 0.1, N)
        x6 = np.random.normal(0.66 * x4, 0.2, N)
        x7 = np.random.normal(0.33 * x4, 0.02, N)
        x8 = np.random.normal(0.20 * x5 + 0.32 * x6 + 0.18 * x7, 0.1, N)
        x9 = np.random.normal(0.5 * x8, 0.03, N)
        x10 = np.random.normal(0.2 * x9, 0.05, N)
        x11 = np.random.normal(0.3 * x9 + 0.4 * x10, 0.03, N)
        x12 = np.random.normal(0.4 * x10 + 0.25 * x11, 0.08, N)
        x13 = np.random.normal(0.2 * x10 + 0.02 * x11 + 0.12 * x12, 0.05, N)
        x14 = np.random.normal(0.5 * x13, 0.01, N)
        x15 = np.random.normal(0.5 * x14, 0.01, N)
        x16 = np.random.normal(0.5 * x15, 0.01, N)
        x17 = np.random.normal(0.5 * x16, 0.01, N)

        data = np.vstack([x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17]).T

        data_normalized = (data - np.min(data)) / np.ptp(data)
        
        df = pd.DataFrame(data_normalized, columns=[f'Feature{i}' for i in range(18)])
        
        return df

In [ ]:
coef = [1, 0.5]
generator = DataGenerator(coef)
df = generator.next(N=100)
df.head()

In [ ]:
#X, y = make_classification(n_samples=100, n_features=18, random_state=42)
#X = norm.ppf((np.abs(X) - X.min()) / (X.max() - X.min()))
#df = pd.DataFrame(np.abs(X), columns=[f'Feature{i}' for i in range(18)])
#df["target"] = y
#df.head()

# Generate Chernoff Faces

In [ ]:
def generate_chernoff_face(ax, x):
    x[3] = 1.9 * (x[3]-.5)
    x[4] = (x[4] + .25)
    x[5] = (x[5] + .2)
    x[6] = .3 * (x[6] + .01)
    x[7] = 5 * (x[7] + .001)
    x[8] /= 5
    x[11] = 2 * (x[11] - .5)
    x[12] += .05
    x[13] += .1
    x[14] = .5 * (x[14] - .5)
    x[15] = .25 * x[15]
    x[16] = .5 * (x[16] - .5)
    x[17] = .5 * (x[17] + .1)
    
    # Face (top)
    ax.add_artist(mpl.Ellipse((0, (x[0] + x[2]) / 2), 2 * x[3], (x[0] - x[2]), fc='white', edgecolor='black', linewidth=2))

    # Face (bottom)
    ax.add_artist(mpl.Ellipse((0, (-x[0] + x[1] + x[2]) / 2), 2 * x[4], (x[0] + x[1] + x[2]), fc='white', edgecolor='black', linewidth=2))

    # Nose
    ax.plot([0, 0], [-x[5] / 2, x[5] / 2], 'k')

    # Mouth
    mouth_arc = mpl.Arc((0, -x[6] + 0.5 / x[7]), 1 / x[7], 1 / x[7],
                                       theta1=270 - 180 / np.pi * np.arctan(x[7] * x[8]),
                                       theta2=270 + 180 / np.pi * np.arctan(x[7] * x[8]))
    ax.add_artist(mouth_arc)

    # Left Eye & Right Eye
    for eye_x in [-x[10] - x[13] / 2, x[10] + x[13] / 2]:
        ax.add_artist(mpl.Ellipse((eye_x, x[9]), x[13], x[12] * x[13], angle=180 / np.pi * x[11], facecolor='white', edgecolor='black'))

    # Left Pupil & Right Pupil
    for pupil_x in [-x[10] - x[13] / 2 - x[14] * x[13] / 2, x[10] + x[13] / 2 - x[14] * x[13] / 2]:
        ax.add_artist(mpl.Ellipse((pupil_x, x[9]), 0.05, 0.05, facecolor='black'))

    # Left Brow & Right Brow
    for brow_x in [-x[10] - x[13] / 2, x[10] + x[13] / 2]:
        ax.plot([brow_x - x[13] * x[17] / 2, brow_x + x[13] * x[17] / 2],
                [x[9] + x[12] * x[13] * (x[15] + x[16]), x[9] + x[12] * x[13] * (x[15] - x[16])], 'k')

# Plot

In [ ]:
target = np.random.randint(0, 2, 100)

In [ ]:
fig, ax = plt.subplots(5,10, figsize=(20,10))
c = 0
for i in range(5):
    for j in range(10):
        data = df.loc[np.random.randint(df.shape[0])]
        if target[c] == 0:
            cface(ax[i, j], data)    
            ax[i, j].set_facecolor('xkcd:green')
        else:
            cface(ax[i, j], data)
            ax[i, j].set_facecolor("xkcd:red")

        c += 1
        ax[i,j].axis([-1.5, 1.5, -1.5, 1.5])
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])